In [1]:
import torch
import torch.nn as nn
import os
import glob
import numpy as np
import torch.utils.data as data
import json
import pandas as pd
import random
import time
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
import math

In [2]:
def get_filenames(data_dir):
    imagelist = []
    p = os.path.join(data_dir, '*.npz')
    imagelist.extend(glob.glob(p))
    return imagelist
 
train_path = 'D:\\MYJF\\data\\payday_npz2'
train_images = get_filenames(train_path)
#train_images = train_images[:50000]
class MyDataset(data.Dataset):
    def __init__(self, images_dir):
        self.images_dir = images_dir
    def __getitem__(self, index):#返回的是tensor
        data_array = np.load(self.images_dir[index])
        update_interval = data_array['arr_1'][2]
#    '是否周末及节假日', '是否工作时间', '通话次数', '主叫通话时长', '被叫通话时长', '未知方式通话时长','和紧急联系人通话',\
#    '和通讯录通话','和高危对象通话', '发送短信次数', '接收短信次数', '未知方式短信次数','和高危对象短信', '4G上网时间', \
#    '3G上网时间', '2G上网时间','占用流量总和'
        if update_interval:
            arr_是否周末及节假日 = np.hstack((np.zeros(update_interval),data_array['arr_0'][:-update_interval,0])).reshape(-1,24)
            arr_是否工作时间 = np.hstack((np.zeros(update_interval),data_array['arr_0'][:-update_interval,1])).reshape(-1,24)
            arr_通话次数 = np.hstack((np.zeros(update_interval),data_array['arr_0'][:-update_interval,2])).reshape(-1,24)
            arr_主叫通话时长 = np.hstack((np.zeros(update_interval),data_array['arr_0'][:-update_interval,3])).reshape(-1,24)
            arr_被叫通话时长 = np.hstack((np.zeros(update_interval),data_array['arr_0'][:-update_interval,4])).reshape(-1,24)
            arr_未知方式通话时长 = np.hstack((np.zeros(update_interval),data_array['arr_0'][:-update_interval,5])).reshape(-1,24)
            arr_和紧急联系人通话 = np.hstack((np.zeros(update_interval),data_array['arr_0'][:-update_interval,6])).reshape(-1,24)
            arr_和通讯录通话 = np.hstack((np.zeros(update_interval),data_array['arr_0'][:-update_interval,7])).reshape(-1,24)
            arr_和高危对象通话 = np.hstack((np.zeros(update_interval),data_array['arr_0'][:-update_interval,8])).reshape(-1,24)
            arr_发送短信次数 = np.hstack((np.zeros(update_interval),data_array['arr_0'][:-update_interval,9])).reshape(-1,24)
            arr_接收短信次数 = np.hstack((np.zeros(update_interval),data_array['arr_0'][:-update_interval,10])).reshape(-1,24)
            arr_未知方式短信次数 = np.hstack((np.zeros(update_interval),data_array['arr_0'][:-update_interval,11])).reshape(-1,24)
            arr_和高危对象短信 = np.hstack((np.zeros(update_interval),data_array['arr_0'][:-update_interval,12])).reshape(-1,24)
            arr_4G上网时间 = np.hstack((np.zeros(update_interval),data_array['arr_0'][:-update_interval,13])).reshape(-1,24)
            arr_3G上网时间 = np.hstack((np.zeros(update_interval),data_array['arr_0'][:-update_interval,14])).reshape(-1,24)
            arr_2G上网时间 = np.hstack((np.zeros(update_interval),data_array['arr_0'][:-update_interval,15])).reshape(-1,24)
            arr_占用流量总和 = np.hstack((np.zeros(update_interval),data_array['arr_0'][:-update_interval,16])).reshape(-1,24)
        else:
            arr_是否周末及节假日 = data_array['arr_0'][:,0].reshape(-1,24)
            arr_是否工作时间 = data_array['arr_0'][:,1].reshape(-1,24)
            arr_通话次数 = data_array['arr_0'][:,2].reshape(-1,24)
            arr_主叫通话时长 = data_array['arr_0'][:,3].reshape(-1,24)
            arr_被叫通话时长 = data_array['arr_0'][:,4].reshape(-1,24)
            arr_未知方式通话时长 = data_array['arr_0'][:,5].reshape(-1,24)
            arr_和紧急联系人通话 = data_array['arr_0'][:,6].reshape(-1,24)
            arr_和通讯录通话 = data_array['arr_0'][:,7].reshape(-1,24)
            arr_和高危对象通话 = data_array['arr_0'][:,8].reshape(-1,24)
            arr_发送短信次数 = data_array['arr_0'][:,9].reshape(-1,24)
            arr_接收短信次数 = data_array['arr_0'][:,10].reshape(-1,24)
            arr_未知方式短信次数 = data_array['arr_0'][:,11].reshape(-1,24)
            arr_和高危对象短信 = data_array['arr_0'][:,12].reshape(-1,24)
            arr_4G上网时间 = data_array['arr_0'][:,13].reshape(-1,24)
            arr_3G上网时间 = data_array['arr_0'][:,14].reshape(-1,24)
            arr_2G上网时间 = data_array['arr_0'][:,15].reshape(-1,24)
            arr_占用流量总和 = data_array['arr_0'][:,16].reshape(-1,24)
            
        arr_是否周末及节假日 = np.hstack((
            arr_是否周末及节假日, np.vstack((arr_是否周末及节假日[1:,:4], np.array([[0,0,0,0]])))))
        arr_是否工作时间 = np.hstack((
            arr_是否工作时间, np.vstack((arr_是否工作时间[1:,:4], np.array([[0,0,0,0]])))))
        arr_通话次数 = np.hstack((
            arr_通话次数, np.vstack((arr_通话次数[1:,:4], np.array([[0,0,0,0]])))))
        arr_主叫通话时长 = np.hstack((
            arr_主叫通话时长, np.vstack((arr_主叫通话时长[1:,:4], np.array([[0,0,0,0]])))))
        arr_被叫通话时长 = np.hstack((
            arr_被叫通话时长, np.vstack((arr_被叫通话时长[1:,:4], np.array([[0,0,0,0]])))))
        arr_未知方式通话时长 = np.hstack((
            arr_未知方式通话时长, np.vstack((arr_未知方式通话时长[1:,:4], np.array([[0,0,0,0]])))))
        arr_和紧急联系人通话 = np.hstack((
            arr_和紧急联系人通话, np.vstack((arr_和紧急联系人通话[1:,:4], np.array([[0,0,0,0]])))))
        arr_和通讯录通话 = np.hstack((
            arr_和通讯录通话, np.vstack((arr_和通讯录通话[1:,:4], np.array([[0,0,0,0]])))))
        arr_和高危对象通话 = np.hstack((
            arr_和高危对象通话, np.vstack((arr_和高危对象通话[1:,:4], np.array([[0,0,0,0]])))))
        arr_发送短信次数 = np.hstack((
            arr_发送短信次数, np.vstack((arr_发送短信次数[1:,:4], np.array([[0,0,0,0]])))))
        arr_接收短信次数 = np.hstack((
            arr_接收短信次数, np.vstack((arr_接收短信次数[1:,:4], np.array([[0,0,0,0]])))))
        arr_未知方式短信次数 = np.hstack((
            arr_未知方式短信次数, np.vstack((arr_未知方式短信次数[1:,:4], np.array([[0,0,0,0]])))))
        arr_和高危对象短信 = np.hstack((
            arr_和高危对象短信, np.vstack((arr_和高危对象短信[1:,:4], np.array([[0,0,0,0]])))))
        arr_4G上网时间 = np.hstack((
            arr_4G上网时间, np.vstack((arr_4G上网时间[1:,:4], np.array([[0,0,0,0]])))))
        arr_3G上网时间 = np.hstack((
            arr_3G上网时间, np.vstack((arr_3G上网时间[1:,:4], np.array([[0,0,0,0]])))))
        arr_2G上网时间 = np.hstack((
            arr_2G上网时间, np.vstack((arr_2G上网时间[1:,:4], np.array([[0,0,0,0]])))))
        arr_占用流量总和 = np.hstack((
            arr_占用流量总和, np.vstack((arr_占用流量总和[1:,:4], np.array([[0,0,0,0]])))))/100000
        arr_label = (data_array['arr_1'][0][:1] > 2)
        
        return np.array([arr_是否周末及节假日[:112,:],arr_是否工作时间[:112,:], arr_通话次数[:112,:],arr_主叫通话时长[:112,:],\
                         arr_被叫通话时长[:112,:],arr_未知方式通话时长[:112,:],arr_和紧急联系人通话[:112,:],\
                         arr_和通讯录通话[:112,:],arr_和高危对象通话[:112,:],arr_发送短信次数[:112,:],\
                        arr_接收短信次数[:112,:],arr_未知方式短信次数[:112,:],arr_和高危对象短信[:112,:]
                        ]).astype('float32'),arr_label.astype('float32')

    def __len__(self):
        return len(self.images_dir)
random_st = random.choice(range(10000))
# train_images, test_images = train_test_split(train_images,test_size=0.15, random_state=random_st)

#跨时间排序
train_images.sort()
split = math.floor(0.85*len(train_images))
train_images2 = train_images
train_images = train_images2[:split]
test_images = train_images2[split:]
print(len(train_images))
print(len(test_images))

train_data = MyDataset(train_images)
test_data = MyDataset(test_images)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=160, shuffle=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=30, shuffle=False, num_workers=0)

85055
15010


In [5]:
import torch
import torch.nn as nn
import os
import glob
import numpy as np
import torch.utils.data as data
import json
import pandas as pd
import random
import time
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
import math

from torch.autograd import Variable
import torch.nn as nn
import torch
import math


def Conv_3x3(inp, oup, stride):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
        nn.ReLU6(inplace=True)
    )


def Conv_1x1(inp, oup):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm2d(oup),
        nn.ReLU6(inplace=True)
    )

def SepConv_3x3(inp, oup): #input=32, output=16
    return nn.Sequential(
        # dw
        nn.Conv2d(inp, inp , 3, 1, 1, groups=inp, bias=False),
        nn.BatchNorm2d(inp),
        nn.ReLU6(inplace=True),
        # pw-linear
        nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm2d(oup),
    )


class InvertedResidual(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio, kernel): # input_channel, output_channel, s, t, k
        super(InvertedResidual, self).__init__()
        self.stride = stride
#         assert stride in [1, 2]

        self.use_res_connect = self.stride == 1 and inp == oup

        self.conv = nn.Sequential(
            # pw
            nn.Conv2d(inp, inp * expand_ratio, 1, 1, 0, bias=False),
            nn.BatchNorm2d(inp * expand_ratio),
            nn.ReLU6(inplace=True),
            # dw
            nn.Conv2d(inp * expand_ratio, inp * expand_ratio, kernel, stride, kernel // 2, 
                      groups=inp * expand_ratio, bias=False),
            nn.BatchNorm2d(inp * expand_ratio),
            nn.ReLU6(inplace=True),
            # pw-linear
            nn.Conv2d(inp * expand_ratio, oup, 1, 1, 0, bias=False),
            nn.BatchNorm2d(oup),
        )

    def forward(self, x):
        if self.use_res_connect:
            return x + self.conv(x)
        else:
            return self.conv(x)


class MnasNet(nn.Module):
    def __init__(self, n_class=1, input_size=224, width_mult=1.):
        super(MnasNet, self).__init__()

        # setting of inverted residual blocks
        self.interverted_residual_setting = [
            # t, c, n, s, k
            [3, 24,  3, 2, 3],  # -> 56x56
            [3, 40,  2, (2,1), 5],  # -> 28x28
            [6, 80,  2, (2,1), 5],  # -> 14x14
            [6, 96,  2, 1, 3],  # -> 14x14
            [6, 192, 4, 2, 5],  # -> 7x7
            [6, 320, 1, 1, 3],  # -> 7x7
        ]
        
#         assert input_size % 32 == 0
        input_channel = int(32 * width_mult)
        self.last_channel = int(1280 * width_mult) if width_mult > 1.0 else 1280
        self.features = [nn.BatchNorm2d(13)]
        input_channel = 16
        self.features .append(Conv_1x1(13, input_channel))

        # building first two layer
#         self.features = [Conv_3x3(3, input_channel, 2), SepConv_3x3(input_channel, 16)]

        # building inverted residual blocks (MBConv)
        for t, c, n, s, k in self.interverted_residual_setting:
            output_channel = int(c * width_mult)
            for i in range(n):
                if i == 0:
                    self.features.append(InvertedResidual(input_channel, output_channel, s, t, k))
                else:
                    self.features.append(InvertedResidual(input_channel, output_channel, 1, t, k))
                input_channel = output_channel

        # building last several layers
        self.features.append(Conv_1x1(input_channel, self.last_channel))
        self.features.append(nn.AvgPool2d(7))

        # make it nn.Sequential
        self.features = nn.Sequential(*self.features)
        self.sigmoid = nn.Sigmoid()
        # building classifier
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(self.last_channel, n_class),
        )

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, self.last_channel)
        x = self.classifier(x)
        x = self.sigmoid(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                n = m.weight.size(1)
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()


net = MnasNet()
x_image = torch.randn([1, 13, 112, 28])
y = net(x_image)
print(y.size())

torch.Size([1, 1])


In [6]:
MnasNet()

MnasNet(
  (features): Sequential(
    (0): BatchNorm2d(13, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Sequential(
      (0): Conv2d(13, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace)
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2d(16, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace)
        (3): Conv2d(48, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=48, bias=False)
        (4): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU6(inplace)
        (6): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (7): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (

In [7]:
model = MnasNet()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device) 
criterion = nn.BCELoss()
opt = torch.optim.Adam(model.parameters())

In [8]:
kss = []
start = time.time()
total_step = len(train_loader)
total_step_test = len(test_loader)
num_epochs = 11
ks_record_train = []
ks_record_test = []
auc_record_train = []
auc_record_test = []
for epoch in range(num_epochs):
    epoch_start = time.time()
    train_label = []
    train_pred = []
    model.train()
    
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        out = model(images)
        loss = criterion(out, labels)
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        print('train epoch: {}/{}, round: {}/{}, loss: {}'.format(epoch + 1, num_epochs,  i + 1, total_step, loss))
        train_label.extend(labels.cpu().numpy().flatten().tolist())
        train_pred.extend(out.detach().cpu().numpy().flatten().tolist())
        
    fpr_lm_train, tpr_lm_train, _ = roc_curve(np.array(train_label), np.array(train_pred))
    ks = abs(fpr_lm_train - tpr_lm_train).max()
    auc = metrics.auc(fpr_lm_train, tpr_lm_train)
    ks_record_train.append(ks)
    auc_record_train.append(auc)
    print('train epoch: {}/{}, KS: {}, ROC: {}'.format(
        epoch + 1, num_epochs, ks, auc))
    
    test_label = []
    test_pred = []
    model.eval()
    for i, (images, labels) in enumerate(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        out = model(images)
        loss = criterion(out, labels)
        print('test epoch: {}/{}, round: {}/{}, loss: {}'.format(epoch + 1, num_epochs,  i + 1, total_step_test, loss))
        test_label.extend(labels.cpu().numpy().flatten().tolist())
        test_pred.extend(out.detach().cpu().numpy().flatten().tolist())
    fpr_lm_test, tpr_lm_test, _ = roc_curve(np.array(test_label), np.array(test_pred))
    ks = abs(fpr_lm_test - tpr_lm_test).max()
    auc = metrics.auc(fpr_lm_test, tpr_lm_test)
    ks_record_test.append(ks)
    auc_record_test.append(auc)
    print('test epoch: {}/{}, KS: {}, ROC: {}'.format(
        epoch + 1, num_epochs, ks, auc))
    print('cost time: {}'.format(int(time.time() - epoch_start)))
    
    # save each model
    kss.append(ks)
    torch.save(model, 'model_{}.pkl'.format(epoch))
end = time.time()
elapsed = end-start
print("{:.3f}ms per batch".format(elapsed/100 * 1000))

train epoch: 1/11, round: 1/532, loss: 0.6095359325408936
train epoch: 1/11, round: 2/532, loss: 0.46186384558677673
train epoch: 1/11, round: 3/532, loss: 0.47775769233703613
train epoch: 1/11, round: 4/532, loss: 0.39920419454574585
train epoch: 1/11, round: 5/532, loss: 0.40526729822158813
train epoch: 1/11, round: 6/532, loss: 0.6033232808113098
train epoch: 1/11, round: 7/532, loss: 0.47736233472824097
train epoch: 1/11, round: 8/532, loss: 0.4120855927467346
train epoch: 1/11, round: 9/532, loss: 0.362922728061676
train epoch: 1/11, round: 10/532, loss: 0.38715535402297974
train epoch: 1/11, round: 11/532, loss: 0.4622225761413574
train epoch: 1/11, round: 12/532, loss: 0.42515382170677185
train epoch: 1/11, round: 13/532, loss: 0.37365466356277466
train epoch: 1/11, round: 14/532, loss: 0.4789457321166992
train epoch: 1/11, round: 15/532, loss: 0.5316734910011292
train epoch: 1/11, round: 16/532, loss: 0.3837811350822449
train epoch: 1/11, round: 17/532, loss: 0.4096271395683288

train epoch: 1/11, round: 139/532, loss: 0.43623486161231995
train epoch: 1/11, round: 140/532, loss: 0.4004620611667633
train epoch: 1/11, round: 141/532, loss: 0.3993839621543884
train epoch: 1/11, round: 142/532, loss: 0.4587554335594177
train epoch: 1/11, round: 143/532, loss: 0.5141721367835999
train epoch: 1/11, round: 144/532, loss: 0.45599016547203064
train epoch: 1/11, round: 145/532, loss: 0.44086846709251404
train epoch: 1/11, round: 146/532, loss: 0.37754273414611816
train epoch: 1/11, round: 147/532, loss: 0.43546780943870544
train epoch: 1/11, round: 148/532, loss: 0.4786175787448883
train epoch: 1/11, round: 149/532, loss: 0.47482091188430786
train epoch: 1/11, round: 150/532, loss: 0.4914519786834717
train epoch: 1/11, round: 151/532, loss: 0.38753122091293335
train epoch: 1/11, round: 152/532, loss: 0.4053500294685364
train epoch: 1/11, round: 153/532, loss: 0.45560646057128906
train epoch: 1/11, round: 154/532, loss: 0.3770368695259094
train epoch: 1/11, round: 155/53

train epoch: 1/11, round: 275/532, loss: 0.3994697034358978
train epoch: 1/11, round: 276/532, loss: 0.30715593695640564
train epoch: 1/11, round: 277/532, loss: 0.38704913854599
train epoch: 1/11, round: 278/532, loss: 0.4943629801273346
train epoch: 1/11, round: 279/532, loss: 0.3579765856266022
train epoch: 1/11, round: 280/532, loss: 0.38371139764785767
train epoch: 1/11, round: 281/532, loss: 0.49297958612442017
train epoch: 1/11, round: 282/532, loss: 0.34391817450523376
train epoch: 1/11, round: 283/532, loss: 0.40104445815086365
train epoch: 1/11, round: 284/532, loss: 0.3499120771884918
train epoch: 1/11, round: 285/532, loss: 0.33998608589172363
train epoch: 1/11, round: 286/532, loss: 0.3663008511066437
train epoch: 1/11, round: 287/532, loss: 0.4244020879268646
train epoch: 1/11, round: 288/532, loss: 0.374237596988678
train epoch: 1/11, round: 289/532, loss: 0.5125390887260437
train epoch: 1/11, round: 290/532, loss: 0.3740819990634918
train epoch: 1/11, round: 291/532, lo

train epoch: 1/11, round: 411/532, loss: 0.4511660039424896
train epoch: 1/11, round: 412/532, loss: 0.35550037026405334
train epoch: 1/11, round: 413/532, loss: 0.4955320358276367
train epoch: 1/11, round: 414/532, loss: 0.3792484700679779
train epoch: 1/11, round: 415/532, loss: 0.42499858140945435
train epoch: 1/11, round: 416/532, loss: 0.434422105550766
train epoch: 1/11, round: 417/532, loss: 0.46905532479286194
train epoch: 1/11, round: 418/532, loss: 0.3397729992866516
train epoch: 1/11, round: 419/532, loss: 0.4094770848751068
train epoch: 1/11, round: 420/532, loss: 0.346376895904541
train epoch: 1/11, round: 421/532, loss: 0.43188023567199707
train epoch: 1/11, round: 422/532, loss: 0.5684941411018372
train epoch: 1/11, round: 423/532, loss: 0.36820974946022034
train epoch: 1/11, round: 424/532, loss: 0.4058305621147156
train epoch: 1/11, round: 425/532, loss: 0.3941088318824768
train epoch: 1/11, round: 426/532, loss: 0.3895835280418396
train epoch: 1/11, round: 427/532, lo

test epoch: 1/11, round: 15/501, loss: 0.4855014681816101
test epoch: 1/11, round: 16/501, loss: 0.5286858677864075
test epoch: 1/11, round: 17/501, loss: 0.4037284851074219
test epoch: 1/11, round: 18/501, loss: 0.5574264526367188
test epoch: 1/11, round: 19/501, loss: 0.5890697240829468
test epoch: 1/11, round: 20/501, loss: 0.7816969752311707
test epoch: 1/11, round: 21/501, loss: 0.40253162384033203
test epoch: 1/11, round: 22/501, loss: 0.6200827360153198
test epoch: 1/11, round: 23/501, loss: 0.5653849244117737
test epoch: 1/11, round: 24/501, loss: 0.404579758644104
test epoch: 1/11, round: 25/501, loss: 0.5836800336837769
test epoch: 1/11, round: 26/501, loss: 0.6757588982582092
test epoch: 1/11, round: 27/501, loss: 0.22583205997943878
test epoch: 1/11, round: 28/501, loss: 0.44953230023384094
test epoch: 1/11, round: 29/501, loss: 0.3210379183292389
test epoch: 1/11, round: 30/501, loss: 0.5834327936172485
test epoch: 1/11, round: 31/501, loss: 0.47706857323646545
test epoch:

...

train epoch: 11/11, round: 439/532, loss: 0.23563846945762634
train epoch: 11/11, round: 440/532, loss: 0.32183483242988586
train epoch: 11/11, round: 441/532, loss: 0.3062857985496521
train epoch: 11/11, round: 442/532, loss: 0.3904515504837036
train epoch: 11/11, round: 443/532, loss: 0.29814213514328003
train epoch: 11/11, round: 444/532, loss: 0.36400556564331055
train epoch: 11/11, round: 445/532, loss: 0.3456394672393799
train epoch: 11/11, round: 446/532, loss: 0.4290781021118164
train epoch: 11/11, round: 447/532, loss: 0.3131128251552582
train epoch: 11/11, round: 448/532, loss: 0.44577592611312866
train epoch: 11/11, round: 449/532, loss: 0.32580453157424927
train epoch: 11/11, round: 450/532, loss: 0.3021591305732727
train epoch: 11/11, round: 451/532, loss: 0.3574487864971161
train epoch: 11/11, round: 452/532, loss: 0.32064276933670044
train epoch: 11/11, round: 453/532, loss: 0.42190995812416077
train epoch: 11/11, round: 454/532, loss: 0.36237093806266785
train epoch: 11

test epoch: 11/11, round: 41/501, loss: 0.4042665660381317
test epoch: 11/11, round: 42/501, loss: 0.3796650767326355
test epoch: 11/11, round: 43/501, loss: 0.3392970860004425
test epoch: 11/11, round: 44/501, loss: 0.5845600962638855
test epoch: 11/11, round: 45/501, loss: 0.6405214071273804
test epoch: 11/11, round: 46/501, loss: 0.5037298202514648
test epoch: 11/11, round: 47/501, loss: 0.25352743268013
test epoch: 11/11, round: 48/501, loss: 0.5227482914924622
test epoch: 11/11, round: 49/501, loss: 0.3155593276023865
test epoch: 11/11, round: 50/501, loss: 0.2465369999408722
test epoch: 11/11, round: 51/501, loss: 0.47862404584884644
test epoch: 11/11, round: 52/501, loss: 0.3927147686481476
test epoch: 11/11, round: 53/501, loss: 0.579917848110199
test epoch: 11/11, round: 54/501, loss: 0.6118279695510864
test epoch: 11/11, round: 55/501, loss: 0.29200229048728943
test epoch: 11/11, round: 56/501, loss: 0.40150752663612366
test epoch: 11/11, round: 57/501, loss: 0.30668768286705

test epoch: 11/11, round: 179/501, loss: 0.2764410078525543
test epoch: 11/11, round: 180/501, loss: 0.29713425040245056
test epoch: 11/11, round: 181/501, loss: 0.5829992890357971
test epoch: 11/11, round: 182/501, loss: 0.5832056403160095
test epoch: 11/11, round: 183/501, loss: 0.4692832827568054
test epoch: 11/11, round: 184/501, loss: 0.6192670464515686
test epoch: 11/11, round: 185/501, loss: 0.4604606032371521
test epoch: 11/11, round: 186/501, loss: 0.5717681646347046
test epoch: 11/11, round: 187/501, loss: 0.5385062098503113
test epoch: 11/11, round: 188/501, loss: 0.47446316480636597
test epoch: 11/11, round: 189/501, loss: 0.5144062638282776
test epoch: 11/11, round: 190/501, loss: 0.4893653988838196
test epoch: 11/11, round: 191/501, loss: 0.3843797445297241
test epoch: 11/11, round: 192/501, loss: 0.6020694971084595
test epoch: 11/11, round: 193/501, loss: 0.5989486575126648
test epoch: 11/11, round: 194/501, loss: 0.38252246379852295
test epoch: 11/11, round: 195/501, lo

test epoch: 11/11, round: 315/501, loss: 0.27311375737190247
test epoch: 11/11, round: 316/501, loss: 0.30337899923324585
test epoch: 11/11, round: 317/501, loss: 0.31848543882369995
test epoch: 11/11, round: 318/501, loss: 0.3925429880619049
test epoch: 11/11, round: 319/501, loss: 0.5041639804840088
test epoch: 11/11, round: 320/501, loss: 0.3972015380859375
test epoch: 11/11, round: 321/501, loss: 0.35850995779037476
test epoch: 11/11, round: 322/501, loss: 0.48416435718536377
test epoch: 11/11, round: 323/501, loss: 0.46007126569747925
test epoch: 11/11, round: 324/501, loss: 0.3085686266422272
test epoch: 11/11, round: 325/501, loss: 0.4097965359687805
test epoch: 11/11, round: 326/501, loss: 0.5187082290649414
test epoch: 11/11, round: 327/501, loss: 0.5827713012695312
test epoch: 11/11, round: 328/501, loss: 0.19415010511875153
test epoch: 11/11, round: 329/501, loss: 0.3969644010066986
test epoch: 11/11, round: 330/501, loss: 0.4260938763618469
test epoch: 11/11, round: 331/501

test epoch: 11/11, round: 451/501, loss: 0.3373474180698395
test epoch: 11/11, round: 452/501, loss: 0.4088172912597656
test epoch: 11/11, round: 453/501, loss: 0.18053951859474182
test epoch: 11/11, round: 454/501, loss: 0.2127797156572342
test epoch: 11/11, round: 455/501, loss: 0.4944216310977936
test epoch: 11/11, round: 456/501, loss: 0.3313247561454773
test epoch: 11/11, round: 457/501, loss: 0.2372807413339615
test epoch: 11/11, round: 458/501, loss: 0.26587170362472534
test epoch: 11/11, round: 459/501, loss: 0.1454218477010727
test epoch: 11/11, round: 460/501, loss: 0.11061026155948639
test epoch: 11/11, round: 461/501, loss: 0.12498220056295395
test epoch: 11/11, round: 462/501, loss: 0.11028016358613968
test epoch: 11/11, round: 463/501, loss: 0.1254880577325821
test epoch: 11/11, round: 464/501, loss: 0.13463075459003448
test epoch: 11/11, round: 465/501, loss: 0.13386224210262299
test epoch: 11/11, round: 466/501, loss: 0.12897342443466187
test epoch: 11/11, round: 467/50

In [9]:
kss

[0.1477622471608091,
 0.1637916358667652,
 0.1731648858723652,
 0.2096319691776986,
 0.19467516370614282,
 0.20109684982341386,
 0.19462289720672893,
 0.19072269635403838,
 0.1768366074561895,
 0.18368986552576366,
 0.1765368217488371]

In [14]:
# # 保存和加载整个模型
# torch.save(model, 'model.pkl')
model = torch.load('model_3.pkl')

In [15]:
test_label = []
test_pred = []
# model.eval()
for i, (images, labels) in enumerate(test_loader):
    images = images.to(device)
    labels = labels.to(device)
    out = model(images)
    loss = criterion(out, labels)
    print('test epoch: {}/{}, round: {}/{}, loss: {}'.format(epoch + 1, num_epochs,  i + 1, total_step_test, loss))
    test_label.extend(labels.cpu().numpy().flatten().tolist())
    test_pred.extend(out.detach().cpu().numpy().flatten().tolist())

test epoch: 11/11, round: 1/501, loss: 0.3973984122276306
test epoch: 11/11, round: 2/501, loss: 0.2994649112224579
test epoch: 11/11, round: 3/501, loss: 0.2328028827905655
test epoch: 11/11, round: 4/501, loss: 0.3724810779094696
test epoch: 11/11, round: 5/501, loss: 0.39677804708480835
test epoch: 11/11, round: 6/501, loss: 0.3484899401664734
test epoch: 11/11, round: 7/501, loss: 0.4631138741970062
test epoch: 11/11, round: 8/501, loss: 0.4428720474243164
test epoch: 11/11, round: 9/501, loss: 0.5974291563034058
test epoch: 11/11, round: 10/501, loss: 0.6457154154777527
test epoch: 11/11, round: 11/501, loss: 0.216862291097641
test epoch: 11/11, round: 12/501, loss: 0.35410892963409424
test epoch: 11/11, round: 13/501, loss: 0.32568931579589844
test epoch: 11/11, round: 14/501, loss: 0.31599894165992737
test epoch: 11/11, round: 15/501, loss: 0.4862963855266571
test epoch: 11/11, round: 16/501, loss: 0.47851887345314026
test epoch: 11/11, round: 17/501, loss: 0.4092242419719696
te

test epoch: 11/11, round: 139/501, loss: 0.5683423280715942
test epoch: 11/11, round: 140/501, loss: 0.441366970539093
test epoch: 11/11, round: 141/501, loss: 0.2920878827571869
test epoch: 11/11, round: 142/501, loss: 0.6085051894187927
test epoch: 11/11, round: 143/501, loss: 0.3984758257865906
test epoch: 11/11, round: 144/501, loss: 0.43620580434799194
test epoch: 11/11, round: 145/501, loss: 0.33844202756881714
test epoch: 11/11, round: 146/501, loss: 0.6454355716705322
test epoch: 11/11, round: 147/501, loss: 0.46240293979644775
test epoch: 11/11, round: 148/501, loss: 0.5067057013511658
test epoch: 11/11, round: 149/501, loss: 0.37359854578971863
test epoch: 11/11, round: 150/501, loss: 0.6029193997383118
test epoch: 11/11, round: 151/501, loss: 0.4489237368106842
test epoch: 11/11, round: 152/501, loss: 0.5084524154663086
test epoch: 11/11, round: 153/501, loss: 0.6027807593345642
test epoch: 11/11, round: 154/501, loss: 0.58971107006073
test epoch: 11/11, round: 155/501, loss

test epoch: 11/11, round: 275/501, loss: 0.30187708139419556
test epoch: 11/11, round: 276/501, loss: 0.3446420133113861
test epoch: 11/11, round: 277/501, loss: 0.3874594271183014
test epoch: 11/11, round: 278/501, loss: 0.6420825123786926
test epoch: 11/11, round: 279/501, loss: 0.31419387459754944
test epoch: 11/11, round: 280/501, loss: 0.1672978699207306
test epoch: 11/11, round: 281/501, loss: 0.14619861543178558
test epoch: 11/11, round: 282/501, loss: 0.23954330384731293
test epoch: 11/11, round: 283/501, loss: 0.24017122387886047
test epoch: 11/11, round: 284/501, loss: 0.3733217120170593
test epoch: 11/11, round: 285/501, loss: 0.4870239198207855
test epoch: 11/11, round: 286/501, loss: 0.33702611923217773
test epoch: 11/11, round: 287/501, loss: 0.5459907650947571
test epoch: 11/11, round: 288/501, loss: 0.21611028909683228
test epoch: 11/11, round: 289/501, loss: 0.3393224775791168
test epoch: 11/11, round: 290/501, loss: 0.3367862105369568
test epoch: 11/11, round: 291/501

test epoch: 11/11, round: 411/501, loss: 0.42277559638023376
test epoch: 11/11, round: 412/501, loss: 0.4124056398868561
test epoch: 11/11, round: 413/501, loss: 0.5048304796218872
test epoch: 11/11, round: 414/501, loss: 0.34697067737579346
test epoch: 11/11, round: 415/501, loss: 0.3871397078037262
test epoch: 11/11, round: 416/501, loss: 0.4115300476551056
test epoch: 11/11, round: 417/501, loss: 0.23815512657165527
test epoch: 11/11, round: 418/501, loss: 0.29875048995018005
test epoch: 11/11, round: 419/501, loss: 0.3703180253505707
test epoch: 11/11, round: 420/501, loss: 0.3106139004230499
test epoch: 11/11, round: 421/501, loss: 0.3943749666213989
test epoch: 11/11, round: 422/501, loss: 0.47324231266975403
test epoch: 11/11, round: 423/501, loss: 0.6822034120559692
test epoch: 11/11, round: 424/501, loss: 0.4543965756893158
test epoch: 11/11, round: 425/501, loss: 0.32222387194633484
test epoch: 11/11, round: 426/501, loss: 0.5167838335037231
test epoch: 11/11, round: 427/501,

In [16]:
test_pred

[0.044049546122550964,
 0.2345128208398819,
 0.10399012267589569,
 0.047530170530080795,
 0.10066536068916321,
 0.1304420679807663,
 0.07475595921278,
 0.08165820688009262,
 0.04309720918536186,
 0.17516040802001953,
 0.07249382138252258,
 0.10393200069665909,
 0.09334211051464081,
 0.13484381139278412,
 0.13223084807395935,
 0.06460276991128922,
 0.17493398487567902,
 0.09255404770374298,
 0.0909789428114891,
 0.08940394967794418,
 0.07806077599525452,
 0.18272213637828827,
 0.1020895317196846,
 0.034803878515958786,
 0.15620091557502747,
 0.04620646312832832,
 0.11799341440200806,
 0.03598358854651451,
 0.12071331590414047,
 0.08005756884813309,
 0.418770432472229,
 0.15280479192733765,
 0.11832275241613388,
 0.04518819972872734,
 0.09281561523675919,
 0.11084058880805969,
 0.09233279526233673,
 0.04216582700610161,
 0.2027457356452942,
 0.10715745389461517,
 0.09536615759134293,
 0.2607821822166443,
 0.04116865247488022,
 0.18536357581615448,
 0.1475083827972412,
 0.1986937075853347

In [17]:
d = {'test_label': test_label, 'test_pred': test_pred}
df = pd.DataFrame(data=d)
df

,test_label,test_pred
0,0.0,0.044050
1,1.0,0.234513
2,0.0,0.103990
3,0.0,0.047530
4,0.0,0.100665
5,0.0,0.130442
6,1.0,0.074756
7,0.0,0.081658
8,0.0,0.043097
9,0.0,0.175160


In [18]:
df.shape

(15010, 2)

In [19]:
df.to_csv("Mnasnet_test_res.csv",index = False)